In [ ]:
from textsum.summarize import Summarizer

summarizer = Summarizer(
    #model_name_or_path="pszemraj/pegasus-x-large-book_synthsumm",
    #model_name_or_path="pszemraj/long-t5-tglobal-base-16384-book-summary",
    model_name_or_path="pszemraj/led-large-book-summary",
	num_beams=4,
    force_cache=True,
    min_length=16,
    no_repeat_ngram_size=3,
    encoder_no_repeat_ngram_size=3,
    repetition_penalty=3.5,
    early_stopping=True,
    #optimum_onnx=True,
    compile_model=True
)

summarizer.model.half()

In [2]:
import html2text
from cleantext import clean

# Configurar o conversor
converter = html2text.HTML2Text()
converter.ignore_links = True
converter.ignore_images = True
converter.ignore_tables = True

def cleartext(decoded_html):
	texto_limpo = converter.handle(decoded_html)
	texto_limpo = clean(
		texto_limpo,
		fix_unicode=True,
		to_ascii=False,
		lower=False,
		no_line_breaks=True,
		no_urls=True,
		no_emails=True,
		no_phone_numbers=True,
		no_numbers=False,
		no_digits=False,
		no_currency_symbols=True,
		no_punct=False,  # Defina como True se quiser remover pontuação
		replace_with_punct="",
		replace_with_url="",
		replace_with_email="",
		replace_with_phone_number="",
		replace_with_number="",
		replace_with_digit="",
		replace_with_currency_symbol=""
	)

	return texto_limpo

In [3]:
from email.header import decode_header

def decode_mime_header(header):
    """
    Decodifica um cabeçalho MIME codificado conforme RFC 2047.

    Args:
        header (str): O cabeçalho a ser decodificado.

    Returns:
        str: O cabeçalho decodificado como string Unicode.
    """
    if header is None:
        return ""
    decoded_fragments = decode_header(header)
    decoded_string = ""
    for fragment, encoding in decoded_fragments:
        if isinstance(fragment, bytes):
            if encoding:
                try:
                    decoded_string += fragment.decode(encoding, errors='replace')
                except Exception:
                    decoded_string += fragment.decode('utf-8', errors='replace')
            else:
                # Se a codificação não for especificada, tenta utf-8
                decoded_string += fragment.decode('utf-8', errors='replace')
        else:
            decoded_string += fragment
    return decoded_string

In [4]:
import random
from datetime import timedelta
from email.utils import parsedate_to_datetime

def date_format(date_str):
	if date_str:
		try:
			# Parse a string de data usando email.utils
			date = parsedate_to_datetime(date_str)
			
			# Gera um número aleatório de microssegundos para adicionar (entre 0 e 999999)
			random_microseconds = random.randint(0, 999999)
			
			# Adiciona os microssegundos aleatórios à data
			date += timedelta(microseconds=random_microseconds)
			
			# Formata a data para ISO 8601 (Aceito pelo DuckDB)
			return date.isoformat()
		except Exception as e:
			print(f"Erro ao converter a data: {e}")
			return None
	else:
		return None

In [5]:
import mailbox
import chardet

# Caminho para o diretório Maildir
maildir_path = '/home/jadson/.thunderbird/7xvrd8c3.default-release/ImapMail/outlook.office365.com/Antigos n&AOM-o lidos/'

# Abrir o Maildir
mbox = mailbox.Maildir(maildir_path, factory=None)

In [8]:
import duckdb
from tqdm import tqdm

# Conectar ou criar o banco de dados DuckDB
conn = duckdb.connect('emails2.duckdb')

In [9]:
# Criar a tabela de e-mails se não existir
conn.execute("""
CREATE TABLE IF NOT EXISTS emails (
    subject TEXT,
    sender TEXT,
    body TEXT,
    date TIMESTAMP PRIMARY KEY
)
""")

In [10]:
# Iterar sobre as mensagens
for idx, message in enumerate(tqdm(mbox, desc="Processando Mensagens"), 1):
    subject = decode_mime_header(message['subject'])
    sender = decode_mime_header(message['from'])
    
	# Obter a data do e-mail, se disponível
    date = date_format(message.get('date'))
    
    body = ''

    if message.is_multipart():
        for part in message.walk():
            content_type = part.get_content_type()
            if content_type == 'text/plain':
                payload = part.get_payload(decode=True)
                if payload:
                    # Detectar a codificação usando chardet
                    result = chardet.detect(payload)
                    encoding = result['encoding'] if result['encoding'] else 'utf-8'
                    try:
                        decoded_text = payload.decode(encoding, errors='replace')
                        body += cleartext(decoded_text)
                    except Exception as e:
                        print(f"Erro ao decodificar a parte da mensagem: {e}")
            elif content_type == 'text/html':
                payload = part.get_payload(decode=True)
                if payload:
                    # Detectar a codificação usando chardet
                    result = chardet.detect(payload)
                    encoding = result['encoding'] if result['encoding'] else 'utf-8'
                    try:
                        decoded_html = payload.decode(encoding, errors='replace')
                        # Converter HTML para texto usando html2text
                        converted_text = cleartext(decoded_html)
                        body += converted_text
                    except Exception as e:
                        print(f"Erro ao converter HTML da mensagem: {e}")
    else:
        payload = message.get_payload(decode=True)
        if payload:
            # Detectar a codificação usando chardet
            result = chardet.detect(payload)
            encoding = result['encoding'] if result['encoding'] else 'utf-8'
            try:
                decoded_text = payload.decode(encoding, errors='replace')
                body += cleartext(decoded_text)
            except Exception as e:
                print(f"Erro ao decodificar a mensagem: {e}")

    content = summarizer.summarize_string(body)
    
    # Inserir os dados no DuckDB
    try:
        conn.execute("""
        INSERT INTO emails (subject, sender, body, date)
        VALUES (?, ?, ?, ?)
        """, (subject, sender, content, date))
    except Exception as e:
        print(f"Erro ao inserir dados no DuckDB: {e}")

Processando Mensagens:   0%|          | 0/22340 [00:00<?, ?it/s]

Generating Summaries:   0%|          | 0/2 [00:00<?, ?it/s]

10/23/2024 09:49:28 INFO Note: NumExpr detected 20 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
10/23/2024 09:49:28 INFO NumExpr defaulting to 8 threads.
Processando Mensagens:   0%|          | 1/22340 [00:09<57:05:06,  9.20s/it]

Generating Summaries:   0%|          | 0/1 [00:00<?, ?it/s]

Processando Mensagens:   0%|          | 2/22340 [00:11<31:56:00,  5.15s/it]

...

Generating Summaries:   0%|          | 0/2 [00:00<?, ?it/s]

Processando Mensagens:   0%|          | 56/22340 [04:08<32:02:33,  5.18s/it]

Generating Summaries:   0%|          | 0/2 [00:00<?, ?it/s]

Processando Mensagens:   0%|          | 56/22340 [04:11<27:49:31,  4.50s/it]


KeyboardInterrupt: 

In [7]:
# Fechar a conexão com o banco de dados
conn.close()